In [10]:
#import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from pyvis.network import Network
plt.rcParams["figure.figsize"] = (20,10)

In [126]:
df=pd.read_csv('processedData1.csv')
df.head()


,gT,sId,rId,sAcc,rAcc,TranAmount,TranType,TranStatus,sBalbefore,sBalAfter,rBalBefore,rBalAfter,sf1,sf2,sTD,rTD,TranTS,sType,rType
0,N-RegDep,PN_Ret2,PN_EU_0_261,RAcc2,EUAcc0_261,131926.49,Dt,SU,1.000000e+09,9.998681e+08,1.001319e+08,1.000000e+08,True,True,1/6/2011 0:11:22,1/6/2011 0:11:22,1/6/2011 0:11:22,RET,EU
1,N_Reg_RC,PN_EU_1_502,operator,EUAcc1_502,A0,2054.46,ArRC,SU,1.000000e+08,9.999795e+07,9.918004e+07,9.917798e+07,True,True,1/6/2011 0:16:8,1/6/2011 0:16:8,1/6/2011 0:16:8,EU,operator
2,N-RegDep,PN_Ret2,PN_EU_1_362,RAcc2,EUAcc1_362,182548.74,Dt,SU,9.998681e+08,9.996855e+08,1.001825e+08,1.000000e+08,True,True,1/6/2011 0:36:47,1/6/2011 0:36:47,1/6/2011 0:36:47,RET,EU
3,N_Reg_RC,PN_EU_1_26,operator,EUAcc1_26,A0,5507.28,ArRC,SU,1.000000e+08,9.999449e+07,9.918554e+07,9.918004e+07,True,True,1/6/2011 0:36:59,1/6/2011 0:36:59,1/6/2011 0:36:59,EU,operator
4,N_Reg_RC,PN_EU_1_18,operator,EUAcc1_18,A0,2102.60,ArRC,SU,1.000000e+08,9.999790e+07,9.918765e+07,9.918554e+07,True,True,1/6/2011 1:1:26,1/6/2011 1:1:26,1/6/2011 1:1:26,EU,operator


In [99]:
#pandas series into list
senders=list(df['sAcc'])
receivers=list(df['rAcc'])
Amount=list(df['TranAmount'])
l=len(senders)
un=df['sAcc'].nunique()
un2=df['rAcc'].nunique()
print(l,un,un2)

54222 1864 1562


In [39]:
#Define a graph visualise it for basic analysis
net=Network('500px', '500px')
G=nx.DiGraph()

for i in range(1000):
    G.add_edge(senders[i],receivers[i],weight=Amount[i])
net.from_nx(G)
net.show('transactions.html')

In [100]:
#Now from the graph G2 and perform the analysis
net2=Network()
G2=nx.Graph()
for i in range(l):
    G2.add_edge(senders[i],receivers[i],weight=Amount[i])
net2.from_nx(G2)
print(G2)

Graph with 2009 nodes and 6988 edges


In [102]:
#get those nodes whose degree is higher, if no of transactions is high it can be a poosible fraud
nds=list(G2.nodes())
print(type(nds))
degrees = [val for (node, val) in G2.degree()]
l2=len(degrees)

repeated_nodes=[]
count=0
for i in range(l2):
    val=degrees[i]
    if val>=5:
        ind = nds[i]
        repeated_nodes.append(ind)
print(len(repeated_nodes))

<class 'list'>
800


In [127]:
#select only those rows that appeares in repeated nodes
sAccount=df['sAcc']
numSamples=len(sAccount)
count=0
drop_indices=[]
for i in range(numSamples):
    acc=sAccount[i]
    if acc not in repeated_nodes:
        count=count+1
        drop_indices.append(i)

In [131]:
df2=df.drop(drop_indices).reset_index()
df2

,index,gT,sId,rId,sAcc,rAcc,TranAmount,TranType,TranStatus,sBalbefore,sBalAfter,rBalBefore,rBalAfter,sf1,sf2,sTD,rTD,TranTS,sType,rType
0,0,N-RegDep,PN_Ret2,PN_EU_0_261,RAcc2,EUAcc0_261,131926.49,Dt,SU,1.000000e+09,9.998681e+08,1.001319e+08,1.000000e+08,True,True,1/6/2011 0:11:22,1/6/2011 0:11:22,1/6/2011 0:11:22,RET,EU
1,2,N-RegDep,PN_Ret2,PN_EU_1_362,RAcc2,EUAcc1_362,182548.74,Dt,SU,9.998681e+08,9.996855e+08,1.001825e+08,1.000000e+08,True,True,1/6/2011 0:36:47,1/6/2011 0:36:47,1/6/2011 0:36:47,RET,EU
2,3,N_Reg_RC,PN_EU_1_26,operator,EUAcc1_26,A0,5507.28,ArRC,SU,1.000000e+08,9.999449e+07,9.918554e+07,9.918004e+07,True,True,1/6/2011 0:36:59,1/6/2011 0:36:59,1/6/2011 0:36:59,EU,operator
3,7,N-RegC2C,PN_EU_2_5,PN_EU_1_121,EUAcc2_5,EUAcc1_121,66197.31,Ind,SU,1.000000e+08,9.993314e+07,1.000662e+08,1.000000e+08,True,True,1/6/2011 1:17:5,1/6/2011 1:17:5,1/6/2011 1:17:5,EU,EU
4,9,N-RegDep,PN_Ret3,PN_EU_2_86,RAcc3,EUAcc2_86,100475.45,Dt,SU,1.000000e+09,9.998995e+08,1.001005e+08,1.000000e+08,True,True,1/6/2011 1:59:29,1/6/2011 1:59:29,1/6/2011 1:59:29,RET,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32522,54211,N-RegC2C,PN_EU_1_340,PN_EU_2_134,EUAcc1_340,EUAcc2_134,54725.23,Ind,SU,1.002481e+08,1.001928e+08,1.025615e+08,1.025068e+08,True,True,30/9/2011 23:6:31,30/9/2011 23:6:31,30/9/2011 23:6:31,EU,EU
32523,54212,N-RegDep,PN_Ret4,PN_EU_1_171,RAcc4,EUAcc1_171,104825.17,Dt,SU,8.476986e+08,8.475938e+08,1.020491e+08,1.019443e+08,True,True,30/9/2011 23:7:52,30/9/2011 23:7:52,30/9/2011 23:7:52,RET,EU
32524,54219,N-RegDep,PN_Ret6,PN_EU_1_143,RAcc6,EUAcc1_143,213240.93,Dt,SU,8.785127e+08,8.782994e+08,1.008969e+08,1.006837e+08,True,True,30/9/2011 23:33:23,30/9/2011 23:33:23,30/9/2011 23:33:23,RET,EU
32525,54220,N-RegC2C,PN_EU_1_197,PN_EU_2_85,EUAcc1_197,EUAcc2_85,199431.78,Ind,SU,9.755577e+07,9.735435e+07,1.021736e+08,1.019741e+08,True,True,30/9/2011 23:38:5,30/9/2011 23:38:5,30/9/2011 23:38:5,EU,EU


In [136]:
#Now perform Decision tree to find out the transactions 'Ind' or 'Withdrawl Type'
tType=df2['TranType']
dind2=[]
for i in range(len(tType)):
    el=tType[i]
    if el=='ArRC' or el=='Dt':
        dind2.append(i)

In [151]:
df3=df2.drop(dind2).reset_index()
df3

,level_0,index,gT,sId,rId,sAcc,rAcc,TranAmount,TranType,TranStatus,...,sBalAfter,rBalBefore,rBalAfter,sf1,sf2,sTD,rTD,TranTS,sType,rType
0,3,7,N-RegC2C,PN_EU_2_5,PN_EU_1_121,EUAcc2_5,EUAcc1_121,66197.31,Ind,SU,...,9.993314e+07,1.000662e+08,1.000000e+08,True,True,1/6/2011 1:17:5,1/6/2011 1:17:5,1/6/2011 1:17:5,EU,EU
1,6,11,N-RegC2C,PN_EU_1_91,PN_EU_0_213,EUAcc1_91,EUAcc0_213,157137.75,Ind,SU,...,9.984129e+07,1.001571e+08,1.000000e+08,True,True,1/6/2011 2:1:41,1/6/2011 2:1:41,1/6/2011 2:1:41,EU,EU
2,11,17,N_RegWith,PN_EU_1_83,PN_Ret5,EUAcc1_83,RAcc5,75914.76,Wl,SU,...,9.992333e+07,9.998482e+08,9.997719e+08,True,True,1/6/2011 2:25:51,1/6/2011 2:25:51,1/6/2011 2:25:51,EU,RET
3,16,25,N-RegC2C,PN_EU_1_217,PN_EU_0_759,EUAcc1_217,EUAcc0_759,5995.06,Ind,SU,...,9.999394e+07,1.000060e+08,1.000000e+08,True,True,1/6/2011 3:44:35,1/6/2011 3:44:35,1/6/2011 3:44:35,EU,EU
4,21,37,N-RegC2C,PN_EU_3_10,PN_EU_0_1180,EUAcc3_10,EUAcc0_1180,48162.83,Ind,SU,...,9.995136e+07,1.000482e+08,1.000000e+08,True,True,1/6/2011 5:18:19,1/6/2011 5:18:19,1/6/2011 5:18:19,EU,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11527,32516,54202,N_RegWith,PN_EU_1_56,PN_Ret5,EUAcc1_56,RAcc5,58663.59,Wl,SU,...,9.825929e+07,9.066821e+08,9.066231e+08,True,True,30/9/2011 22:48:39,30/9/2011 22:48:39,30/9/2011 22:48:39,EU,RET
11528,32517,54204,N_RegWith,PN_EU_3_33,PN_Ret3,EUAcc3_33,RAcc3,231559.46,Wl,SU,...,1.000289e+08,8.619040e+08,8.616712e+08,True,True,30/9/2011 22:54:3,30/9/2011 22:54:3,30/9/2011 22:54:3,EU,RET
11529,32518,54205,N-RegC2C,PN_EU_1_368,PN_EU_3_3,EUAcc1_368,EUAcc3_3,226206.39,Ind,SU,...,9.691174e+07,9.849312e+07,9.826692e+07,True,True,30/9/2011 22:57:3,30/9/2011 22:57:3,30/9/2011 22:57:3,EU,EU
11530,32522,54211,N-RegC2C,PN_EU_1_340,PN_EU_2_134,EUAcc1_340,EUAcc2_134,54725.23,Ind,SU,...,1.001928e+08,1.025615e+08,1.025068e+08,True,True,30/9/2011 23:6:31,30/9/2011 23:6:31,30/9/2011 23:6:31,EU,EU


In [152]:
#Now perform binary decision tree to find out the transaction with lower than average transaction Amount
tAmt=df3['TranAmount']
l3=len(tAmt)
avg=np.mean(tAmt)
print('Average:',avg)
dind3=[]
#remove all those that have transaction amount higher than average
for i in range(l3):
    amount=tAmt[i]
    if amount>=avg:
        dind3.append(i)

Average: 90643.74966094371


In [153]:
len(dind3)

4080

In [159]:
df4=df3.drop(dind3).reset_index(drop=True)
df4

,level_0,index,gT,sId,rId,sAcc,rAcc,TranAmount,TranType,TranStatus,...,sBalAfter,rBalBefore,rBalAfter,sf1,sf2,sTD,rTD,TranTS,sType,rType
0,3,7,N-RegC2C,PN_EU_2_5,PN_EU_1_121,EUAcc2_5,EUAcc1_121,66197.31,Ind,SU,...,9.993314e+07,1.000662e+08,1.000000e+08,True,True,1/6/2011 1:17:5,1/6/2011 1:17:5,1/6/2011 1:17:5,EU,EU
1,11,17,N_RegWith,PN_EU_1_83,PN_Ret5,EUAcc1_83,RAcc5,75914.76,Wl,SU,...,9.992333e+07,9.998482e+08,9.997719e+08,True,True,1/6/2011 2:25:51,1/6/2011 2:25:51,1/6/2011 2:25:51,EU,RET
2,16,25,N-RegC2C,PN_EU_1_217,PN_EU_0_759,EUAcc1_217,EUAcc0_759,5995.06,Ind,SU,...,9.999394e+07,1.000060e+08,1.000000e+08,True,True,1/6/2011 3:44:35,1/6/2011 3:44:35,1/6/2011 3:44:35,EU,EU
3,21,37,N-RegC2C,PN_EU_3_10,PN_EU_0_1180,EUAcc3_10,EUAcc0_1180,48162.83,Ind,SU,...,9.995136e+07,1.000482e+08,1.000000e+08,True,True,1/6/2011 5:18:19,1/6/2011 5:18:19,1/6/2011 5:18:19,EU,EU
4,22,41,N-RegC2C,PN_EU_1_138,PN_EU_0_500,EUAcc1_138,EUAcc0_500,7203.14,Ind,SU,...,9.999272e+07,1.000034e+08,9.999623e+07,True,True,1/6/2011 5:56:6,1/6/2011 5:56:6,1/6/2011 5:56:6,EU,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7447,32511,54195,N_RegWith,PN_EU_1_145,PN_Ret6,EUAcc1_145,RAcc6,50732.88,Wl,SU,...,9.935678e+07,8.786779e+08,8.786269e+08,True,True,30/9/2011 22:31:1,30/9/2011 22:31:1,30/9/2011 22:31:1,EU,RET
7448,32512,54196,N_RegWith,PN_EU_1_133,PN_Ret1,EUAcc1_133,RAcc1,79774.59,Wl,SU,...,9.872020e+07,8.604540e+08,8.603738e+08,True,True,30/9/2011 22:35:17,30/9/2011 22:35:17,30/9/2011 22:35:17,EU,RET
7449,32513,54198,N-RegC2C,PN_EU_1_217,PN_EU_0_759,EUAcc1_217,EUAcc0_759,44241.30,Ind,SU,...,1.003329e+08,1.010859e+08,1.010416e+08,True,True,30/9/2011 22:40:43,30/9/2011 22:40:43,30/9/2011 22:40:43,EU,EU
7450,32516,54202,N_RegWith,PN_EU_1_56,PN_Ret5,EUAcc1_56,RAcc5,58663.59,Wl,SU,...,9.825929e+07,9.066821e+08,9.066231e+08,True,True,30/9/2011 22:48:39,30/9/2011 22:48:39,30/9/2011 22:48:39,EU,RET


In [161]:
#Now visualize the all transactions along with fraud transactions
#pandas series into list
fse=list(df4['sAcc'])
fre=list(df4['rAcc'])
Amnt=list(df4['TranAmount'])

In [183]:
#Define a graph visualise the fraud transactions
net2=Network('500px', '500px')
G2=nx.DiGraph()
#G2.add_edge(1,3,color='red')
#G2.add_edge(1,2)
for i in range(1000):
    G2.add_edge(senders[i],receivers[i],weight=Amount[i])

for j in range(200):
    G2.add_edge(fse[j],fre[j],weight=Amnt[j],color='red')

net2.from_nx(G2)
net2.show('AnalysedGraph.html')